# Corruption

## Overview
This notebook is designed to apply various types of image corruptions to a dataset of images, primarily for the purpose of testing and enhancing the robustness of image processing algorithms. It utilizes custom corruption functions to modify images by introducing different levels of distortions such as blurring, noise, and other effects.

## Key Functions
- `corrupt(x, severity, corruption_name, corruption_number)`: Main function to apply specified corruptions to images. It takes an image `x`, a `severity` level, and either a `corruption_name` or `corruption_number` to select the type of corruption. It returns the corrupted image.
- `gaussian_blur(x, severity)`: A specific corruption function that applies Gaussian blur to an image. The blur intensity is controlled by the `severity` parameter.

## Corruption Types
The notebook supports several types of image corruptions, including:
- **Gaussian Blur**: Smoothens the image by using a Gaussian function, creating the effect of viewing the image through a translucent screen.
- **(Additional types can be listed as they are implemented or planned)**

## Usage
Users can specify the severity of corruption and the type of corruption to apply to each image. The notebook processes images stored in a specified directory structure, applies the requested corruption, and saves the corrupted images in a new directory structure for further analysis or use.

## Objective
The goal of this notebook is to provide a tool for researchers and developers to generate corrupted image datasets that can be used to evaluate and improve the performance of computer vision models under various noise conditions.


## Imports

In [98]:
import sys
import os
#from imagenet_c import corrupt
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
from skimage.filters import gaussian
import cv2

In [86]:
def gaussian_blur(x, severity=1):
    c = [1, 2, 3, 4, 6][severity - 1]

    # Update the gaussian function call
    x = gaussian(np.array(x) / 255., sigma=c, channel_axis=-1)
    return np.clip(x, 0, 1) * 255

In [93]:
def apply_colormap(x, severity=1):
    # Lista de colourmaps disponibles en OpenCV
    colormaps = [
        cv2.COLORMAP_AUTUMN,
        #cv2.COLORMAP_BONE,
        #cv2.COLORMAP_JET,
        #cv2.COLORMAP_WINTER,
        #cv2.COLORMAP_RAINBOW
    ]

    # Asegurar que el índice de severity es válido
    colormap_index = colormaps[severity - 1] if severity - 1 < len(colormaps) else colormaps[0]

    # Aplicar el colormap seleccionado
    colored_image = cv2.applyColorMap(np.uint8(x), colormap_index)

    return colored_image

In [94]:
def corrupt(x, severity=1, corruption_name=None, corruption_number=-1):
    # Define a dictionary of corruption functions
    corruption_dict = {
        'gaussian_blur': gaussian_blur,  
        'apply_colormap': apply_colormap,
    }

    if corruption_name:
        if corruption_name in corruption_dict:
            x_corrupted = corruption_dict[corruption_name](Image.fromarray(x.astype('uint8')), severity)
        else:
            raise ValueError(f"Corruption name {corruption_name} is not supported.")
    elif corruption_number != -1:
        # Define the tuple if using indices to reference corruptions
        corruption_tuple = (
            gaussian_blur,  # Ensure the tuple is in sync with the corruption names
            # shot_noise,
            # impulse_noise,
            # ... other corruptions ...
        )
        x_corrupted = corruption_tuple[corruption_number](Image.fromarray(x.astype('uint8')), severity)
    else:
        raise ValueError("Either corruption_name or corruption_number must be passed")

    return np.uint8(x_corrupted)

In [95]:
corruptions=['apply_colormap']

In [96]:
# Create directories for corrupted images
for severity in range(1, 6):
    for corruption in corruptions:
        new_dir = f"./Vision+/test_s_{severity}_corr={corruption}"
        if not os.path.exists(new_dir):
            os.makedirs(new_dir)
        for emotion in os.listdir(source_path):
            if not os.path.exists(os.path.join(new_dir, emotion)):
                os.makedirs(os.path.join(new_dir, emotion))

In [99]:
for severity in range(1, 6):
    for corruption in corruptions:
        new_dir = f"./Vision+/test_s_{severity}_corr={corruption}"
        if not os.path.exists(new_dir):
            os.makedirs(new_dir)
        for emotion in os.listdir(source_path):
            emotion_dir = os.path.join(source_path, emotion)
            for img_name in os.listdir(emotion_dir):
                img_path = os.path.join(emotion_dir, img_name)
                img = Image.open(img_path)
                img_array = np.array(img)
                corrupted_img = corrupt(img_array, severity=severity, corruption_name=corruption)
                # Save the corrupted image
                corrupted_img_pil = Image.fromarray(corrupted_img)
                corrupted_img_pil.save(os.path.join(new_dir, emotion, img_name))

print("Process completed.")

KeyboardInterrupt: 